In [1]:
concatenated_nwb = r'C:\Users\guru\Desktop\test_nwb\m201821_20181018_B1-00000.nwb'
# Still missing how to get motor data

In [2]:
electrode_name = 'e0'
description = 'FHC_bipolar electrode'
location = 'Vg'
device_name = 'AM Systems 1700 B277 modified with headstage'
num_sites = 1.
x=1.5
y=0.
z=5.7
imp=2.0
filtering = '300-10000 Hz'
motor_motion_name = 'Port_A_8_bits'
neural_channel_name = 'Neural'

In [3]:
import nwb_wrapper
import pynwb
from pynwb import NWBHDF5IO
import numpy as np
import awake_nwb
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [4]:
with NWBHDF5IO(concatenated_nwb,'r+') as io:# will want to change this to a w+ I think 
    data = io.read()

    acquisition_list = [x.lower() for x in data.acquisition.keys()]
    acquisition_list
    device = data.create_device(name=device_name)

    electrode_group = data.create_electrode_group(electrode_name,description=description,location=location,device=device)
    data.add_electrode(0,
                       x=x,
                       y=y,
                       z=z,
                       imp=imp,
                       location=location,
                       filtering=filtering,
                       group=electrode_group)

    electrode_table_region = data.create_electrode_table_region([0],'The only electrode')
    neural = nwb_wrapper.convert_TS_to_ES(data.get_acquisition(neural_channel_name),electrode_table_region)
    data.add_acquisition(neural)
    if 'sent_trig' in acquisition_list:
        front_samps = awake_nwb.trigger_to_idx(data.get_acquisition('Front'))
        front_times = data.get_acquisition('Front').timestamps.value[front_samps]

        top_samps = awake_nwb.trigger_to_idx(data.get_acquisition('Top'))
        top_times = data.get_acquisition('Top').timestamps.value[top_samps]

        frame_samps = awake_nwb.trigger_to_idx(data.get_acquisition('Sent_trig'))
        frame_times = data.get_acquisition('Sent_trig').timestamps.value[frame_samps]

        # create timeseries
        front_trig = pynwb.TimeSeries('front frame onsets',
                               front_samps,
                                timestamps=front_times,
                               )
        top_trig = pynwb.TimeSeries('top frame onsets',
                               top_samps,
                                timestamps=top_times,
                               )
        cam_trig = pynwb.TimeSeries('camera frame onsets',
                               frame_samps,
                                timestamps=frame_times,
                               )

        data.add_acquisition(front_trig)
        data.add_acquisition(top_trig)
        data.add_acquisition(cam_trig)

        #TODO: check if front=top=sent
    else:
        frame_samps = awake_nwb.trigger_to_idx(data.get_acquisition('Cam_trig').data)
        frame_times = data.get_acquisition('Cam_trig').timestamps.value[frame_samps]
        cam_trig = pynwb.TimeSeries('camera frame onsets',
                               frame_samps,
                                timestamps=frame_times,
                               )
        data.add_acquisition(cam_trig)
        io.write(data)

Frame rate of camera is calculated to be 742.71 fps



C:\Users\guru\Anaconda2\envs\nwb\lib\site-packages\pynwb\form\build\map.py:882: MissingRequiredWarning: attribute 'unit' for 'data' (TimeSeries)
  warnings.warn(msg, MissingRequiredWarning)


In [6]:
io = NWBHDF5IO(concatenated_nwb,'r')
d2 = io.read()
d2.acquisition
    

{'Cam_trig': 
 Cam_trig <class 'pynwb.base.TimeSeries'>
 Fields:
   comments: no comments
   conversion: 1.0
   data: <HDF5 dataset "data": shape (22372011,), type "<f8">
   description: no description
   interval: 1
   num_samples: 22372011
   resolution: 4.8828125
   timestamps: <HDF5 dataset "timestamps": shape (22372011,), type "<f8">
   timestamps_unit: Seconds
   unit: mV, 'Count': 
 Count <class 'pynwb.misc.AnnotationSeries'>
 Fields:
   comments: no comments
   conversion: 1.0
   data: ['1' '1' '2' '1' '1' '2']
   description: no description
   interval: 1
   num_samples: 6
   resolution: -1.0
   timestamps: <HDF5 dataset "timestamps": shape (6,), type "<f8">
   timestamps_unit: Seconds
   unit: n/a, 'Neural': 
 Neural <class 'pynwb.base.TimeSeries'>
 Fields:
   comments: no comments
   conversion: 1.0
   data: <HDF5 dataset "data": shape (22372011,), type "<f8">
   description: no description
   interval: 1
   num_samples: 22372011
   resolution: 0.6103515625
   timestamps: <H

In [ ]:
reload(awake_nwb)

In [ ]:
io.close()

In [ ]:
%pylab